In [5]:
import numpy as np
# from preprocess_CT_image import load_scan, get_pixels_hu
import matplotlib.pyplot as plt
%matplotlib inline
import os
from skimage.transform import radon, iradon
from PIL import Image

import cv2
# import h5py

'''
Absolute Path of data
'''
abosolute_path_input = '/Volumes/Secrets/Research_DAE/LUNA16-Lung-Nodule-Analysis-2016-Challenge-master/LUNA16Challege/'
abosolute_path_output = '../../'
'''
Relative Path of data
'''
input_path = 'data/preprocess/1-raw_bmp/'
output_path = 'data/preprocess/3-noisy_bmp/'

'''
Real Path
'''
input_path = abosolute_path_output + input_path
output_path = abosolute_path_output + output_path

if not os.path.exists(output_path):
    os.mkdir(output_path)


In [18]:
'''
Add Poisson noise to CT image 
input:
    shape: variable
    type: npy
output:
    shape: not changed
    type: .bmp file
'''
def add_noise_CT(image,voxelSize,theta , N0, mu_water=0.0171):
    epsilon = 5
    
    #now get images to linear attenuation coef (1/mm)
    # slices[0].KVP gives the voltage and from there we can find water attenuation
    #here, 0.0227 is the attenuation coef (1/mm) of water at 50 keV
    image = mu_water*image/1000 + mu_water
    # I get an error using radon transform that asks the values outside the tube to be zero
    image = (image-image.min())
    '''Obtatain Sinogram of the image using radon function'''

    proj = radon(image, theta=theta, circle=True)*voxelSize
    #plot the projections
    #plt.subplot(3,1,3),plt.imshow(sinogram[0],cmap='gray')
       
    '''calculate the amount of Poisson noise for each ray in each projection'''
    N = N0*np.exp(-proj)
    N_noise = np.random.poisson(N)
    proj_noisy_image = -np.log(N_noise/N0)
        
    '''Correct for infs in sinogram'''
    #since sometimes N_noise is <= 0, projs_noise can be inf! We need to correct for
    # this, I will do so here by finding all inf values and replacing them by 
    # -log(epsilon/N0) where eplison is some small number >0 that reflects the
    # smallest possible detected photon count
    
    idx = np.isinf(proj_noisy_image)
    proj_noisy_image[idx] = -np.log(epsilon/N0)/voxelSize
    
    '''reconstruct ct image'''
    # there are 2 ways from here: 
    # 1- iradon on the noisy projection,
    noisy_image = iradon(proj_noisy_image, theta=theta,circle=True)

    #2- iradon the noise and add it to the original image, to avoid messy edges of image
#    projs_noise = proj_noisy_image - proj
#    noise = iradon(projs_noise, theta=theta,circle=True)/voxelSize
#    noisy_image = image + noise
#    
    
    return noisy_image

In [31]:

# path = 'your_file_path'
# image= misc.imread(os.path.join(path,'image.bmp'), flatten= 0)

# N0 = 3.2315E2 # controls the level of noise added to the image
# 根据文献，N0应该是设置成1x10^5
N0 = 1E5
# 因为之前已经resample过了，像素间隔被统一设置成了1，1，1
voxelSize = 1 # in mm
for item in os.scandir(input_path):
    if item.is_dir():
        noisy_dir_path = output_path + item.name + '/'
        if not os.path.exists(noisy_dir_path):
            os.mkdir(noisy_dir_path)
        for bmp in sorted(os.listdir(item.path + '/'),key=lambda x:int(x[:-4])):
            if bmp.endswith('.bmp'):
                image = np.array(Image.open(item.path + '/' + bmp))
                # image = np.expand_dims(image,axis=0)
                # print(image.shape)
                print(bmp)
                theta = np.linspace(0., 360., max(image[0].shape), endpoint=False)
                # noisy_image = np.zeros(image.shape)
                image=image.astype(np.float32)
                noisy_image = add_noise_CT(image,voxelSize,theta,N0,mu_water=0.0171)
                print(image.shape, noisy_image.shape)
                noisy_image = (noisy_image-noisy_image.min())/(noisy_image.max()-noisy_image.min())
                image = (image-image.min())/(image.max()-image.min())
                # save noisy image
                if not os.path.exists(output_path + item.name + '/'):
                    os.mkdir(output_path + item.name + '/')
                # noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')

                cv2.imwrite(output_path + item.name + '/' + bmp, noisy_image*255)

                # show in notebook
                # plt.imshow(image,cmap='gray')
                # plt.show()
                # plt.imshow(noisy_image,cmap='gray')
                # plt.show()


                



0.bmp


/Users/kuzaowuwei/opt/anaconda3/lib/python3.7/site-packages/skimage/transform/radon_transform.py:63: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


(354, 354) (354, 354)
1.bmp
(354, 354) (354, 354)
2.bmp
(354, 354) (354, 354)
3.bmp
(354, 354) (354, 354)
4.bmp
(354, 354) (354, 354)
5.bmp
(354, 354) (354, 354)
6.bmp
(354, 354) (354, 354)
7.bmp
(354, 354) (354, 354)
8.bmp
(354, 354) (354, 354)
9.bmp
(354, 354) (354, 354)
10.bmp
(354, 354) (354, 354)
11.bmp
(354, 354) (354, 354)
12.bmp
(354, 354) (354, 354)
13.bmp
(354, 354) (354, 354)
14.bmp
(354, 354) (354, 354)
15.bmp
(354, 354) (354, 354)
16.bmp
(354, 354) (354, 354)
17.bmp
(354, 354) (354, 354)
18.bmp
(354, 354) (354, 354)
19.bmp
(354, 354) (354, 354)
20.bmp
(354, 354) (354, 354)
21.bmp
(354, 354) (354, 354)
22.bmp
(354, 354) (354, 354)
23.bmp
(354, 354) (354, 354)
24.bmp
(354, 354) (354, 354)
25.bmp
(354, 354) (354, 354)
26.bmp
(354, 354) (354, 354)
27.bmp
(354, 354) (354, 354)
28.bmp
(354, 354) (354, 354)
29.bmp
(354, 354) (354, 354)
30.bmp
(354, 354) (354, 354)
31.bmp
(354, 354) (354, 354)
32.bmp
(354, 354) (354, 354)
33.bmp
(354, 354) (354, 354)
34.bmp
(354, 354) (354, 354)
3

KeyboardInterrupt: 

In [ ]:
image = np.load('../data/process/npy_image/0/100.npy')

image = image.reshape(1,512,512)
print(image.shape)
N0 = 3.2315E2 # controls the level of noise added to the image
voxelSize = 1 # in mm
''' Add noise to CT'''
theta = np.linspace(0., 360., max(image[0].shape), endpoint=False)
noisy_image = np.zeros(image.shape)
for i in range(0,image.shape[0]):
    print(i)
    image=image.astype(np.float32)
    noisy_image[i] = add_noise_CT(image[i],voxelSize,theta,N0,mu_water=0.0171)
    noisy_image[i] = (noisy_image[i]-noisy_image[i].min())/(noisy_image[i].max()-noisy_image[i].min())
    image[i] = (image[i]-image[i].min())/(image[i].max()-image[i].min())

plt.figure()
plt.subplot(3,1,1),plt.imshow(image[0],cmap='gray')
plt.subplot(3,1,2),plt.imshow(noisy_image[0],cmap='gray')
plt.show()